# 指令微调

## 下载数据集

In [8]:
import json
import os
import urllib

def download_and_load_file(file_path, url):
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
file_path = "instruction-data.json"
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
       "/main/ch07/01_main-chapter-code/instruction-data.json")

data = download_and_load_file(file_path, url)

In [11]:
print("amount of data: ", len(data))

amount of data:  1100


In [13]:
print(data[50])
print(data[999])

{'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}
{'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


In [14]:
def format_input(entry):
    instruction = (
        f"以下是一个描述任务的指令。\n"
        f"请写出一个适当完成请求的响应\n\n"
        f"### 指令：\n{entry['instruction']}"
    )
    input_text = (
        f"\n\n###输入:\n{entry['input']}" if entry['input'] else ""
    )
    return instruction + input_text